In [22]:
from dataclasses import dataclass
from pathlib import Path
from typing import Tuple, Union

import cv2
import numpy as np
from PIL import Image, ImageEnhance
from numpy.typing import NDArray


@dataclass
class EffectSettings:
    """写真効果の設定を管理するデータクラス"""
    # 色調整
    red_factor: float = 1.3
    green_factor: float = 1.1
    blue_factor: float = 0.8
    contrast: float = 1.4
    brightness: float = 1.2
    
    apply_light_leak: bool = False

    light_leak_count: int = 3
    light_leak_radius_range: Tuple[int, int] = (100, 300)
    light_leak_intensity_range: Tuple[float, float] = (0.5, 1.0)


class VintagePhotoEffect:
    
    def __init__(self, settings: EffectSettings = EffectSettings()):
        self.settings = settings

    def process_image(self, input_path: Union[str, Path], output_path: Union[str, Path]) -> None:
        image = self._load_image(input_path)
        image = self._adjust_colors(image)
        image = self._adjust_contrast_and_brightness(image)
        image = self._add_noise(image)
        image = self._add_vignette(image)
        
        if self.settings.apply_light_leak:
            image = self._add_light_leak(image)
            
        self._save_image(image, output_path)

    def _load_image(self, path: Union[str, Path]) -> Image.Image:
        return Image.open(path).convert('RGB')

    def _adjust_colors(self, image: Image.Image) -> Image.Image:
        red, green, blue = image.split()
        
        def adjust_channel(channel: Image.Image, factor: float) -> Image.Image:
            return channel.point(lambda p: min(255, p * factor))
        
        adjusted_channels = (
            adjust_channel(red, self.settings.red_factor),
            adjust_channel(green, self.settings.green_factor),
            adjust_channel(blue, self.settings.blue_factor)
        )
        return Image.merge('RGB', adjusted_channels)

    def _adjust_contrast_and_brightness(self, image: Image.Image) -> Image.Image:
        image = ImageEnhance.Contrast(image).enhance(self.settings.contrast)
        image = ImageEnhance.Brightness(image).enhance(self.settings.brightness)
        return image

    def _add_noise(self, image: Image.Image, noise_strength: float = 25.0) -> Image.Image:
        image_np = np.array(image, dtype=np.uint8)
        noise = np.random.normal(0, noise_strength, image_np.shape).astype(np.int16)
        noisy_image = np.clip(image_np + noise, 0, 255).astype(np.uint8)
        return Image.fromarray(noisy_image)

    def _add_vignette(self, image: Image.Image) -> Image.Image:
        image_np = np.array(image, dtype=np.uint8)
        rows, cols = image_np.shape[:2]
        
        kernel_x = cv2.getGaussianKernel(cols, cols / 2)
        kernel_y = cv2.getGaussianKernel(rows, rows / 2)
        kernel = kernel_y * kernel_x.T
        
        mask = 255 * kernel / np.max(kernel)
        vignette = np.dstack([mask] * 3)
        
        result = np.clip(image_np * (vignette / 255), 0, 255).astype(np.uint8)
        return Image.fromarray(result)

    def _add_light_leak(self, image: Image.Image) -> Image.Image:
        image_np = np.array(image, dtype=np.uint8)
        rows, cols = image_np.shape[:2]
        light_leak = np.zeros_like(image_np, dtype=np.uint8)

        for _ in range(self.settings.light_leak_count):
            x, y, radius, intensity = self._generate_light_leak_params(rows, cols)
            mask = self._create_light_leak_mask(rows, cols, x, y, radius, intensity)
            light_leak = self._apply_light_leak_color(light_leak, mask)

        result = np.clip(image_np + light_leak, 0, 255).astype(np.uint8)
        return Image.fromarray(result)

    def _generate_light_leak_params(self, rows: int, cols: int) -> Tuple[int, int, int, float]:
        if np.random.rand() < 0.5:
            x = np.random.randint(0, cols // 2)
            y = np.random.randint(rows // 2, rows)
        else:
            x = np.random.randint(cols // 2, cols)
            y = np.random.randint(0, rows)
        
        min_radius, max_radius = self.settings.light_leak_radius_range
        min_intensity, max_intensity = self.settings.light_leak_intensity_range
        
        radius = np.random.randint(min_radius, max_radius)
        intensity = np.random.uniform(min_intensity, max_intensity)
        return x, y, radius, intensity

    def _create_light_leak_mask(self, rows: int, cols: int, x: int, y: int,
                              radius: int, intensity: float) -> NDArray:
        mask = np.zeros((rows, cols), dtype=np.float32)
        cv2.circle(mask, (x, y), radius, intensity, -1, lineType=cv2.LINE_AA)
        return mask

    def _apply_light_leak_color(self, light_leak: NDArray, mask: NDArray) -> NDArray:
        for c in range(3):
            light_leak[:, :, c] += (mask * np.random.uniform(100, 255)).astype(np.uint8)
        return light_leak

    def _save_image(self, image: Image.Image, path: Union[str, Path]) -> None:
        image.save(path)


def apply_vintage_effect(
    input_path: str,
    output_path: str,
    settings: EffectSettings = EffectSettings()
) -> None:
    effect = VintagePhotoEffect(settings)
    effect.process_image(input_path, output_path)


# 使用例
if __name__ == "__main__":
    # 基本的な使用方法（ライトリークなし）
    apply_vintage_effect('./images/20241225_085106.jpg', 'output_no_leak.jpg')
    
    # ライトリーク効果を有効にした使用例
    settings_with_leak = EffectSettings(
        # 色調整設定
        red_factor=1.3,
        green_factor=1.2,
        blue_factor=0.8,
        contrast=1.4,
        brightness=1.2,
        # ライトリーク設定
        apply_light_leak=False,
        light_leak_count=1,  # ライトリークの数を2つに
        light_leak_radius_range=(150, 250),  # ライトリークのサイズ範囲を調整
        light_leak_intensity_range=(0.6, 0.9)  # 強度範囲を調整
    )
    apply_vintage_effect('./images/20241225_085106.jpg', 'output_with_leak.jpg', settings_with_leak)